# CREAZIONE DATASET DEI TEAMS

In [4]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

import ast
import time
from datetime import datetime

import pygraphviz as pgv

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

In [5]:
cu = db.users.find()
cs = db.shots.find()
cf = db.followers.find()
cl = db.likes.find()
ct = db.teams.find()

In [6]:
db.users.find()[0]

{u'_id': ObjectId('5bb24252e2850523d71d356d'),
 u'bio': u'Web &amp; UX/UI designer arestov.design@gmail.com',
 u'buckets_count': 5,
 u'comments_received_count': 101,
 u'created_at': 1416216907,
 u'followers_count': 1106,
 u'followings_count': 746,
 u'id': 695302,
 u'likes_count': 5456,
 u'likes_received_count': 4872,
 u'location': u'Moscow, Russia',
 u'name': u'Leonid Arestov',
 u'projects_count': 4,
 u'rebounds_received_count': 7,
 u'shots_count': 65,
 u'teams_count': 0,
 u'teams_url': u'https://api.dribbble.com/v1/users/695302/teams',
 u'updated_at': 1513060644,
 u'username': u'arestov_design'}

In [13]:
db.user.find({'id':43059626})[0]

IndexError: no such item for Cursor instance

In [ ]:
res = db.users.find({'teams_count': {'$gt':0}})
res.count()

In [ ]:
res = db.shots.find({'username': 'AdrienDuchateau'})
print len(res[0]['shots'])
i=0
for elem in res[0]['shots']:
    if elem['team'] != None:
        print elem['team']
        break

### CREAZIONE COLLECTION Teams

In [ ]:
def add_team(team):    

    json_elem = json.loads(json.dumps(team))

    try:
        
        d = datetime.strptime(json_elem['created_at'], '%Y-%m-%dT%H:%M:%SZ')
        unixtime = time.mktime(d.timetuple())
        json_elem['created_at'] = int(unixtime)

        d = datetime.strptime(json_elem['updated_at'], '%Y-%m-%dT%H:%M:%SZ')
        unixtime = time.mktime(d.timetuple())
        json_elem['updated_at'] = int(unixtime)
    
    except:
        print 'date error !!'
        
    db.teams.insert_one(json_elem)


In [ ]:
%%time

db.drop_collection('teams')
cs = db.shots.find()

for elem in cs:
    shots = elem['shots']
    for s in shots:
        if s['team'] != None:
            if db.teams.find({'id': s['team']['id']}).count() == 0:
                add_team(s['team'])
    

### Creazione grafi rappresentazione teams


### team - team

In [3]:
def find_teams_in_shots(shots):    
    teams = []
    if len(shots) > 0:
        for s in shots:
            if s['team'] != None:
                if s['team']['username'] not in teams:
                    teams.append(s['team']['username'])

    return teams

In [4]:
def link(g, nodes):
    for a in nodes:
        g.add_node(a)
        for b in nodes:
            if b!=a:
                g.add_node(b) 
                g.add_edge(a,b)

    return g 

In [5]:
%%time

g=pgv.AGraph()


for elem in cs:
    shots = elem['shots']
    teams = find_teams_in_shots(shots)
    g = link(g, teams)


g.write('teamsGraph_test.dot')

CPU times: user 26.6 s, sys: 3.78 s, total: 30.4 s
Wall time: 32 s


In [8]:
print len(g.nodes())
print db.teams.find().count()

temp_teams = []
for t in db.teams.find():
    temp_teams.append(t['username'])

4058
4055


/home/simo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [10]:
for n in g.nodes():
    if n not in temp_teams:
        print n

horizond
MrzDesignStudio
PRFTDigitalLabs


### user - user (solo utenti che appartengono a un team)

In [ ]:
def find_team_users(team):
    users = db.shots.find(
        {'shots': {
        '$all':[{
                '$elemMatch': {'team.username': team}
                }]
            }}
    )
    
    usernames = []
    for u in users:
        usernames.append(u['username'])
        
    return usernames

In [ ]:
%%time
import pygraphviz as pgv

g = pgv.AGraph()

i=0

for team in ct:
    users = find_team_users(team['username'])
    g = link(g, users)

    i=i+1
    print i
    
    
g.write('usersGraph.dot')

In [ ]:
db.shots.find({'username': 'darianedwards'})[0]

### Trovare data di prima pubblicazione di uno user in un team
### Creazione dataset 'user_team_first_shot' dove memorizzo per ogni utente e per ogni team di tale utente quando è stato il primo shots di tale utente in tale team

In [ ]:
%%time

i=0

user_with_shots = db.shots.find({'shots': {'$not': {'$size': 0}}})

for u in user_with_shots:
    teams = find_teams_in_shots(u['shots'])
    
    for t in teams:
        mindate = 10000000000
        for s in u['shots']:
            if s['team'] != None and s['team']['username'] == t:

                if s['created_at'] < mindate:
                    mindate = s['created_at']

             
        #print 'USER: ',u['username']
        #print 'TEAM: ',t
        #print 'MIN: ',mindate
        elem = {'user': u['username'], 'team': t, 'date': mindate}
        
        db.user_team_first_shot.insert_one(elem)
        

            
    i=i+1
    print i
    


In [ ]:
user = db.shots.find({'shots': {'$not': {'$size': 0}}})
user[0]

In [ ]:
cut = db.user_team_first_shot.find()
print cut.count()
for e in cut:
    print e